# Train and fine-tune a decision tree for the moons dataset

# Step 1: Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

In [1]:
!pip install scikit-learn
!pip install graphviz

In [2]:
# import modules
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# generate moons dataset
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# Step 2: Use train_test_split() to split the dataset into a training set and a test set.

In [4]:
# split dataset into training and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print length of training and test set
print("Training set length: ", len(X_train))
print("Test set length: ", len(X_test))

Training set length:  8000
Test set length:  2000


# Step 3: Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

In [5]:
# use GridSearchCV to find the best hyperparameters for a DecisionTreeClassifier
# try various values for max_leaf_nodes
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
]

tree_clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree_clf, param_grid, cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)

Fitting 3 folds for each of 294 candidates, totalling 882 fits
Best hyperparameters:  {'max_leaf_nodes': 17, 'min_samples_split': 2}


# Step 4: Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [6]:
# train on the entire training set with the best hyperparameters
from sklearn.metrics import accuracy_score

final_model = grid_search.best_estimator_

y_pred = final_model.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))

Test set accuracy:  0.8695


# Grow a forest

# Step 1: Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.

In [7]:
# use ShuffleSplit to create 1000 subsets of the training set, each containing 100 instances chosen randomly
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

# Step 2: Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.

In [8]:
# train one decision tree on each subset
# use best hyperparameters from previous step
# evaluate 1000 decision trees on the test set
from sklearn.base import clone

forest = [clone(final_model) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# print the mean accuracy of the 1000 decision trees
print("Mean accuracy of 1000 decision trees: ", np.mean(accuracy_scores))

Mean accuracy of 1000 decision trees:  0.805471


# Step 3: Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.

In [9]:
# generate predictions for 1000 decision trees
# use scipy's mode() function to find the most frequent prediction

from scipy.stats import mode

Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

/tmp/ipykernel_315/725574019.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


# Step 4: Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a random forest classifier!

In [10]:
# evaluate these predictions on the test set
print("Accuracy of majority voting: ", accuracy_score(y_test, y_pred_majority_votes.reshape([-1])))

Accuracy of majority voting:  0.872
